In [0]:
pip install requests

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import requests
import json
from pyspark.sql.types import *
from pyspark.sql.functions import explode, split, current_date

In [0]:
response_data = requests.get('https://reqres.in/api/users?page=2')
json_data = response_data.json()
display(json_data)

{'page': 2,
 'per_page': 6,
 'total': 12,
 'total_pages': 2,
 'data': [{'id': 7,
   'email': 'michael.lawson@reqres.in',
   'first_name': 'Michael',
   'last_name': 'Lawson',
   'avatar': 'https://reqres.in/img/faces/7-image.jpg'},
  {'id': 8,
   'email': 'lindsay.ferguson@reqres.in',
   'first_name': 'Lindsay',
   'last_name': 'Ferguson',
   'avatar': 'https://reqres.in/img/faces/8-image.jpg'},
  {'id': 9,
   'email': 'tobias.funke@reqres.in',
   'first_name': 'Tobias',
   'last_name': 'Funke',
   'avatar': 'https://reqres.in/img/faces/9-image.jpg'},
  {'id': 10,
   'email': 'byron.fields@reqres.in',
   'first_name': 'Byron',
   'last_name': 'Fields',
   'avatar': 'https://reqres.in/img/faces/10-image.jpg'},
  {'id': 11,
   'email': 'george.edwards@reqres.in',
   'first_name': 'George',
   'last_name': 'Edwards',
   'avatar': 'https://reqres.in/img/faces/11-image.jpg'},
  {'id': 12,
   'email': 'rachel.howell@reqres.in',
   'first_name': 'Rachel',
   'last_name': 'Howell',
   'avatar'

In [0]:
data_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("email", StringType(), True),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("avatar", StringType(), True)
])

custom_schema = StructType([
    StructField("page", IntegerType(), True),
    StructField("per_page", IntegerType(), True),
    StructField("total", IntegerType(), True),
    StructField("total_pages", IntegerType(), True),
    StructField("data", ArrayType(data_schema), True),
    StructField("support", MapType(StringType(), StringType()), True)
])


In [0]:
df = spark.createDataFrame([json_data], custom_schema)
display(df)

page,per_page,total,total_pages,data,support
2,6,12,2,"List(List(7, michael.lawson@reqres.in, Michael, Lawson, https://reqres.in/img/faces/7-image.jpg), List(8, lindsay.ferguson@reqres.in, Lindsay, Ferguson, https://reqres.in/img/faces/8-image.jpg), List(9, tobias.funke@reqres.in, Tobias, Funke, https://reqres.in/img/faces/9-image.jpg), List(10, byron.fields@reqres.in, Byron, Fields, https://reqres.in/img/faces/10-image.jpg), List(11, george.edwards@reqres.in, George, Edwards, https://reqres.in/img/faces/11-image.jpg), List(12, rachel.howell@reqres.in, Rachel, Howell, https://reqres.in/img/faces/12-image.jpg))","Map(url -> https://reqres.in/#support-heading, text -> To keep ReqRes free, contributions towards server costs are appreciated!)"


In [0]:
df = df.drop('page', 'per_page', 'total', 'total_pages', 'support')
display(df)

data
"List(List(7, michael.lawson@reqres.in, Michael, Lawson, https://reqres.in/img/faces/7-image.jpg), List(8, lindsay.ferguson@reqres.in, Lindsay, Ferguson, https://reqres.in/img/faces/8-image.jpg), List(9, tobias.funke@reqres.in, Tobias, Funke, https://reqres.in/img/faces/9-image.jpg), List(10, byron.fields@reqres.in, Byron, Fields, https://reqres.in/img/faces/10-image.jpg), List(11, george.edwards@reqres.in, George, Edwards, https://reqres.in/img/faces/11-image.jpg), List(12, rachel.howell@reqres.in, Rachel, Howell, https://reqres.in/img/faces/12-image.jpg))"


In [0]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: integer (nullable = true)
 |    |    |-- email: string (nullable = true)
 |    |    |-- first_name: string (nullable = true)
 |    |    |-- last_name: string (nullable = true)
 |    |    |-- avatar: string (nullable = true)



In [0]:
df = df.withColumn('data', explode('data'))
display(df)

data
"List(7, michael.lawson@reqres.in, Michael, Lawson, https://reqres.in/img/faces/7-image.jpg)"
"List(8, lindsay.ferguson@reqres.in, Lindsay, Ferguson, https://reqres.in/img/faces/8-image.jpg)"
"List(9, tobias.funke@reqres.in, Tobias, Funke, https://reqres.in/img/faces/9-image.jpg)"
"List(10, byron.fields@reqres.in, Byron, Fields, https://reqres.in/img/faces/10-image.jpg)"
"List(11, george.edwards@reqres.in, George, Edwards, https://reqres.in/img/faces/11-image.jpg)"
"List(12, rachel.howell@reqres.in, Rachel, Howell, https://reqres.in/img/faces/12-image.jpg)"


In [0]:
df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- id: integer (nullable = true)
 |    |-- email: string (nullable = true)
 |    |-- first_name: string (nullable = true)
 |    |-- last_name: string (nullable = true)
 |    |-- avatar: string (nullable = true)



In [0]:
df = df.withColumn("id", df.data.id).withColumn('email', df.data.email).withColumn('first_name', df.data.first_name).withColumn('last_name', df.data.last_name).withColumn('aatar', df.data.avatar).drop(df.data)
display(df)

id,email,first_name,last_name,aatar
7,michael.lawson@reqres.in,Michael,Lawson,https://reqres.in/img/faces/7-image.jpg
8,lindsay.ferguson@reqres.in,Lindsay,Ferguson,https://reqres.in/img/faces/8-image.jpg
9,tobias.funke@reqres.in,Tobias,Funke,https://reqres.in/img/faces/9-image.jpg
10,byron.fields@reqres.in,Byron,Fields,https://reqres.in/img/faces/10-image.jpg
11,george.edwards@reqres.in,George,Edwards,https://reqres.in/img/faces/11-image.jpg
12,rachel.howell@reqres.in,Rachel,Howell,https://reqres.in/img/faces/12-image.jpg


In [0]:
derived_site_address_df = df.withColumn("site_address",split(df["email"],"@")[1])
display(derived_site_address_df)

id,email,first_name,last_name,aatar,site_address
7,michael.lawson@reqres.in,Michael,Lawson,https://reqres.in/img/faces/7-image.jpg,reqres.in
8,lindsay.ferguson@reqres.in,Lindsay,Ferguson,https://reqres.in/img/faces/8-image.jpg,reqres.in
9,tobias.funke@reqres.in,Tobias,Funke,https://reqres.in/img/faces/9-image.jpg,reqres.in
10,byron.fields@reqres.in,Byron,Fields,https://reqres.in/img/faces/10-image.jpg,reqres.in
11,george.edwards@reqres.in,George,Edwards,https://reqres.in/img/faces/11-image.jpg,reqres.in
12,rachel.howell@reqres.in,Rachel,Howell,https://reqres.in/img/faces/12-image.jpg,reqres.in


In [0]:
loaded_date = derived_site_address_df.withColumn('load_date', current_date())
display(loaded_date)

id,email,first_name,last_name,aatar,site_address,load_date
7,michael.lawson@reqres.in,Michael,Lawson,https://reqres.in/img/faces/7-image.jpg,reqres.in,2024-04-17
8,lindsay.ferguson@reqres.in,Lindsay,Ferguson,https://reqres.in/img/faces/8-image.jpg,reqres.in,2024-04-17
9,tobias.funke@reqres.in,Tobias,Funke,https://reqres.in/img/faces/9-image.jpg,reqres.in,2024-04-17
10,byron.fields@reqres.in,Byron,Fields,https://reqres.in/img/faces/10-image.jpg,reqres.in,2024-04-17
11,george.edwards@reqres.in,George,Edwards,https://reqres.in/img/faces/11-image.jpg,reqres.in,2024-04-17
12,rachel.howell@reqres.in,Rachel,Howell,https://reqres.in/img/faces/12-image.jpg,reqres.in,2024-04-17


In [0]:
loaded_date.write.format('delta').mode('overwrite').save('dbfs:/FileStore/assignments/question2/site_info/person_info')

In [0]:
testing_df = spark.read.format('delta').load('dbfs:/FileStore/assignments/question2/site_info/person_info')
display(testing_df)

id,email,first_name,last_name,aatar,site_address,load_date
7,michael.lawson@reqres.in,Michael,Lawson,https://reqres.in/img/faces/7-image.jpg,reqres.in,2024-04-17
8,lindsay.ferguson@reqres.in,Lindsay,Ferguson,https://reqres.in/img/faces/8-image.jpg,reqres.in,2024-04-17
9,tobias.funke@reqres.in,Tobias,Funke,https://reqres.in/img/faces/9-image.jpg,reqres.in,2024-04-17
10,byron.fields@reqres.in,Byron,Fields,https://reqres.in/img/faces/10-image.jpg,reqres.in,2024-04-17
11,george.edwards@reqres.in,George,Edwards,https://reqres.in/img/faces/11-image.jpg,reqres.in,2024-04-17
12,rachel.howell@reqres.in,Rachel,Howell,https://reqres.in/img/faces/12-image.jpg,reqres.in,2024-04-17
